
## Project Goal
I will use a dataset for a fictional supermarket chain to practice importing and cleaning excel data and to create a script that can be executed from the command line. 


In [1]:
#Import libraries.
import numpy as np
import pandas as pd
import os

In [2]:
# Create an output folder.
#os.mkdir('output')
#**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists! 

In [3]:
# Create a files variable that contains all of our data files.
files = os.listdir('data')

In [4]:
# Read in data from only ONE city from January 1st.
nytest = pd.read_excel("./data/Jan 1.xlsx", sheet_name = 'New York')

In [5]:
nytest.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   prodcode   129 non-null    int64  
 1   price_eu   129 non-null    float64
 2   weight_kg  129 non-null    float64
 3   quantity   129 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 4.2 KB


In [6]:
#convert to American measurements
nytest.insert(2,'price_usd',(nytest['price_eu'] * 1.1),True)
nytest.insert(4,'weight_lb',(nytest['weight_kg'] * 2.2),True)


In [7]:
nytest

,prodcode,price_eu,price_usd,weight_kg,weight_lb,quantity
0,4159,2.048141,2.252955,8.812961,19.388515,138
1,4062,2.728485,3.001334,9.331372,20.529018,184
2,4404,2.182498,2.400748,8.937798,19.663156,491
3,4412,2.342694,2.576964,7.067682,15.548899,218
4,4131,1.802839,1.983123,9.551503,21.013306,391
...,...,...,...,...,...,...
124,4796,1.827676,2.010444,7.922745,17.430039,444
125,4225,2.463229,2.709552,5.839095,12.846008,258
126,4134,0.920152,1.012167,9.225959,20.297110,395
127,3302,1.510998,1.662098,2.667668,5.868870,168


In [8]:
#merge in product names
plu = pd.read_csv("plu_codes.csv")

plu.head()

,product,plu_code
0,Apple (Fuji),4131
1,Apple (Gala),4134
2,Apricot,3302
3,Avocado,4225
4,Banana,4011


In [9]:
nytest = nytest.join(plu.set_index('plu_code'), on=['prodcode'])

In [10]:
# Drop unnecessary Columns
nytest.drop(columns = 'price_eu', inplace = True)
nytest.drop(columns = 'weight_kg', inplace = True)


In [11]:
# add the Date column
nytest['date'] = 'Jan 1'

In [12]:
nytest


,prodcode,price_usd,weight_lb,quantity,product,date
0,4159,2.252955,19.388515,138,Onion,Jan 1
1,4062,3.001334,20.529018,184,Cucumber,Jan 1
2,4404,2.400748,19.663156,491,Peach,Jan 1
3,4412,2.576964,15.548899,218,Pear,Jan 1
4,4131,1.983123,21.013306,391,Apple (Fuji),Jan 1
...,...,...,...,...,...,...
124,4796,2.010444,17.430039,444,Tomato,Jan 1
125,4225,2.709552,12.846008,258,Avocado,Jan 1
126,4134,1.012167,20.297110,395,Apple (Gala),Jan 1
127,3302,1.662098,5.868870,168,Apricot,Jan 1


In [13]:
#Create a single function to perform the cleaning from the previous cells.

def process_data(file, city):
    
    #Read in the data from the given file and city.
    plu = pd.read_csv("plu_codes.csv")
    sales = pd.read_excel(file, sheet_name = city)
    
    #Create USD and pound columns.
    sales.insert(2,'price_usd',(sales['price_eu'] * 1.1),True)
    sales.insert(4,'weight_lb', (sales['weight_kg'] * 2.2),True)
    
    #Merge in product names.
    sales = sales.join(plu.set_index('plu_code'), on= ['prodcode'])
    
    #Drop unnecessary columns.
    sales.drop(columns = 'price_eu', inplace = True)
    sales.drop(columns = 'weight_kg', inplace = True)
    
    #Add a date column
    def get_date(file_name):
        numbers = '1234567890'
        if file_name[12] in numbers:
            return file_name[7:13]
        else:
            return file_name[7:12]
    sales['date'] = get_date(file)
    return sales


In [14]:
#test the function on a new file
process_data('./data/Jan 12.xlsx', 'New York')

,prodcode,price_usd,weight_lb,quantity,product,date
0,4011,0.684078,19.023640,388,Banana,Jan 12
1,4958,1.938633,9.763554,253,Lemon,Jan 12
2,3302,2.897492,13.344452,407,Apricot,Jan 12
3,4012,0.645215,8.725540,348,Orange,Jan 12
4,4159,1.334039,17.003191,141,Onion,Jan 12
...,...,...,...,...,...,...
105,4030,2.954453,10.743151,431,Kiwi,Jan 12
106,4240,0.626234,19.990259,213,Blueberries,Jan 12
107,4323,1.168074,17.096145,285,Strawberries,Jan 12
108,4272,2.229775,15.239064,371,Grapes,Jan 12


In [15]:
#Process all of the January data for one city.
ny_jan = [process_data('./data/' + file,'New York') for file in files]

In [16]:
#Concatenate all the January DataFrames
ny_jan = pd.concat(ny_jan)

## Perform cleaning on all of the cities for January and write a new csv for each city


In [17]:
#Create a dictionary to write a file for each city. 
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [18]:
# Loop through city_dict to clean and write a csv for each city for January.

for (key,value) in city_dict.items():
    ready = pd.concat([process_data('./data/' + file, key) for file in files])
    ready['city'] = key
    ready = ready.loc[:,["city","date", "product", "prodcode", "quantity", "weight_lb", "price_usd"]]
    ready.to_csv('./output/'+ value)

In [31]:
# to check the work I read the files back in and check the averages for each city

city_files = os.listdir('output')
city_data = pd.concat([pd.read_csv('./output/' + file,index_col=0) for file in city_files])
city_data.groupby(by = 'city').mean()

,prodcode,quantity,weight_lb,price_usd
city,,,,
Atlanta,4221.003568,302.485306,11.929027,1.937980
Austin,4226.061277,301.048298,12.092775,1.937456
Boston,4232.272331,298.806536,12.063057,1.900973
Chicago,4227.265551,301.686186,12.101555,1.930026
Denver,4232.204734,301.012745,12.129850,1.932088
Los Angeles,4226.015120,301.531276,12.167527,1.918331
New York,4224.536969,299.482863,12.090294,1.912662
San Francisco,4224.431414,298.979895,12.237399,1.925830
Seattle,4238.488624,300.333694,11.925486,1.903315


Now I am ready to clean the code up and turn it into an executable script. 
